We extract video frames from selected animations and extract
the line art images to form our training dataset. We calculate a
**768-dimensional feature vector of histograms of R, G, B channels
for each frame**. The difference between frames is determined by
calculating the **mean square error** of the feature vectors, which
is used for splitting the source animations into shots. When the
difference between the neighboring frames is **greater than 200**, it
is considered to belong to different shots. In order to improve the
quality of the data, we **remove shots in which the mean square
errors between all pairs of frames are less than 10** (as they are
too uniform), and **the shot with a length less than 8 frames**.
Then we **filter out video frames that are too dark or too faded
in color**. Finally we get a total of 
1096 video sequences from 6 animations, with a total of 29,834 images. 
Each video sequence
has 27 frames on average. 

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import json
import os
import shutil
import pandas

Let's create the frames.

In [48]:
def VidToFrames_2 (vidpath, folderName):
    vidcap = cv2.VideoCapture(vidpath)
    success,image = vidcap.read()
    print(success)
    count = 0
    length = int(vidcap.get(cv2.CAP_PROP_FPS))
    width  = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(length, width, height)
    while success:
        resized = cv2.resize(image, (256,256), interpolation = cv2.INTER_AREA)
        cv2.imwrite("{}/{}.png".format(foldername= folderName.split('-',1)[-1], frameNum=str(count.zfill(5))), resized)  
        success,image = vidcap.read()
        count += 1
    print(count, "frames in file")

def VidToFrames (vidpath, folderName):
    vidcap = cv2.VideoCapture(vidpath)
    success,image = vidcap.read()
    print(success)
    count = 0
    length = int(vidcap.get(cv2.CAP_PROP_FPS))
    width  = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(length, width, height)
    while success:
        cv2.imwrite("{foldername}/{frameNum}.png".format(foldername= folderName, frameNum=str(count)), image)  
        success,image = vidcap.read()
        count += 1
    print(count, "frames in file")

for i in glob.glob("mp4/*.mp4"):
    print()
    destination = i.split('.mp4')[0]
    os.mkdir(destination)
    VidToFrames_2(i, destination)


True
24 640 360
10105 frames in file


Now that we have folders with frames from each video for each channelId, we can start the analysis. There is two parts to this: 
1. Color analysis
2. Motion analysis

### Part I  <br/>
***Color Analysis*** <br/>
Is there a correlation between color and subcount?

In [78]:
from IPython.display import Image

def removeDimAndFadedImages(folderName):
    sortedFrameList = sorted(glob.glob(folderName+'/*.png'), key=os.path.getmtime)
    debugCount = 0
    for imagePath in sortedFrameList:
        brightness = getBrightnessOfImage(imagePath)
        if brightness < 9.4e7 or brightness > 4.5e8: # brightness < 6*10^7 or brightness > 7*10^8:
            print(imagePath, ": ", brightness)
            os.remove(imagePath)
        if debugCount % 100 == 0: 
            print (debugCount)
        debugCount +=1

        
def getBrightnessOfImage(imagePath):
    image = cv2.imread(imagePath)
    vectors = []
    for i, col in enumerate(['b', 'g', 'r']):
        hist = cv2.calcHist([image], [i], None, [256], [0, 256])
        sums = np.sum(np.fromiter((index*hist[index] for index in range(len(hist))), dtype=float))
        vectors.append(sums)
        plt.plot(hist, color = col)
        plt.xlim([0, 256])
    plt.show()
    totalSums = np.sum(vectors)
    return totalSums

An image histogram gives a graphical representation of the distribution of pixel intensities in a digital image.

In [73]:
def binAndPlot(some_list):
    sorted_list = sorted(some_list)
    sorted_counted = Counter(sorted_list)
    range_length = list(range(int(max(some_list)+1))) # Get the largest value to get the range.
    data_series = {}

    for i in range_length:
        data_series[i] = 0 # Initialize series so that we have a template and we just have to fill in the values.
    for key, value in sorted_counted.items():
        data_series[key] = value
    data_series = pd.Series(data_series)
    x_values = data_series.index
    plt.hist(some_list, edgecolor="yellow", color="green")
    plt.show()

### Part 2:
*** Motion Analysis ***

Is there a correlation between the amount of animation and subcount?

In my head, I should technically analyze how much each frame differs from each other. 
However, color is a value that is assigned to each pixel in a frame.

So we are going to get rid of color. We're not completely getting rid of it tho- any changes due to color will be detected (example)- because we are using
ahem ahem an eXtended difference-of-Gaussians to extract the lines. So we're basically changing the data to put less weight on color because it's already been accounted for in Part I.

 There was a time when I was interested in colorization machine learning techniques. Honestly, I still am, just less than this project. 

 There was a paper called [Deep Line Art Video Colorization with a Few References](https://arxiv.org/abs/2003.10685) that I spent some time generating data for. It was only after I generated all the data that I realized I don't have the resources (GPU, money, honestly, motivation was the biggest factor)

> We extract video frames from selected animations and extract
the line art images to form our training dataset. We calculate a
768-dimensional feature vector of histograms of R, G, B channels
for each frame. The difference between frames is determined by
calculating the mean square error of the feature vectors, which
is used for splitting the source animations into shots. When the
difference between the neighboring frames is greater than 200, it
is considered to belong to different shots. In order to improve the
quality of the data, we remove shots in which the mean square
errors between all pairs of frames are less than 10 (as they are
too uniform), and the shot with a length less than 8 frames.
Then we filter out video frames that are too dark or too faded
in color. Finally we get a total of 1096 video sequences from 6
animations, with a total of 29,834 images. Each video sequence
has 27 frames on average. 

In my case, I got around 28k iumages with 32 frames per scene on average.

I'm going to use the data collection method stated above, with a slight variation


In [24]:
with open('../channelId_to_mostViewedVidId.json', 'r') as j:
    channelId_to_mostViewedVidId = json.loads(j.read())

LIST_OF_CHANNEL_IDS= channelId_to_mostViewedVidId.keys()
print(len(LIST_OF_CHANNEL_IDS))

62


In [28]:
%ls ../

__pycache__/
animation_analysis/
channelHolisticInfo.json
channelId_to_mostViewedVidId.json
channelListResponse.json
client_secret.json
credential_sample.json
dataProcessing.ipynb
dataProcessing.py
listOfVideoIds.json
listOfVideoIds_1.json
main.py
masterSheetOfChannelIdToListOfVidIds.json
masterSheetOfChannelIdToListOfVidInfos.json
mergeJson.py
reddit sentiment.png
searchListResponse.json
semantic_analysis/
transcripts.py
videoListResponse.json
webscraping youtubeapi.zip
webscraping.py


In [32]:
with open('../masterSheetOfChannelIdToListOfVidInfos.json', 'r') as j:
    masterSheet = json.loads(j.read())

channelId_to_mostViewedVidInfo = {}

for channelId in LIST_OF_CHANNEL_IDS:
    mostViewedVidId = channelId_to_mostViewedVidId[channelId]
    for vidInfo in masterSheet[channelId]:
        if vidInfo['vidId'] == mostViewedVidId:
            channelId_to_mostViewedVidInfo[channelId] = vidInfo
channelId_to_mostViewedVidInfo
# print(len(channelId_to_mostViewedVidId))

{'UCsDmESjqNPukDmVnuneLrqw': {'vidId': 'EcgkRp2IUsc',
  'publishTime': '2017-10-26T01:05:22Z',
  'vidTitle': 'My Hot Babysitter Got Me In Trouble',
  'duration': 'PT6M30S',
  'viewCount': 35960829,
  'likeCount': 840561,
  'commentCount': 75550,
  'topicCategories': ['https://en.wikipedia.org/wiki/Entertainment',
   'https://en.wikipedia.org/wiki/Film'],
  'description': "First Name Babysitter. Last Name Trouble. \nGET T-SHIRTS: http://www.itsalexclark.com \nSEE ME ON TOUR: http://www.itsalexclark.com/tour\n\nThanks to Shgurr and PamonSunset:\nhttps://www.youtube.com/channel/UCkQ3eqNNaFrBZ_WbnCIMN8w\n\nSubscribe: http://bit.ly/SubAlexClark | Get ItsAlexClark merch: http://itsalexclark.bigcartel.com/ \nWatch my latest video: https://youtube.com/watch?v=YIiBGDBN-Po&list=PLmh1WGagp73Jbndkw6nCz2JCm73jEp8gh\nHelp out the channel: https://Patreon.com/itsalexclark\n\nFollow ItsAlexClark!\nFacebook: https://facebook.com/itsalexclark/?ref=br_rs\nInstagram: https://instagram.com/itsalexclark/\nT

In [33]:
import regex as re
df = pandas.DataFrame.from_dict(channelId_to_mostViewedVidInfo).T
not_candidate_vidIds= ['kQEtRoyFfI8', 'hpQQohcHk9Q', '6wS_uON5s6Q', 'RlU32AfEVeU', 'iVqhzEaJhDw', 'Xnv7JGqjaAo', '18msRdBF11A', 'Ln4AnsWNUQI', 'Hkz0NcKPzMs']

#https://en.wikipedia.org/wiki/ISO_8601#Durations
for channelId, row in df.iterrows():
    x= row['duration']
    m=0
    s=0
    try:
        m, s = re.findall('PT(\d+)M(\d+)S',x)[0]
    except:
        s = re.findall('PT(\d+)S',x)[0]
    if int(m)*60+int(s) < 210:
        not_candidate_vidIds.append(row['vidId'])
not_candidate_vidIds = set(not_candidate_vidIds)
candidateVideos = list(set(df.vidId)- not_candidate_vidIds)
df = df[df.vidId.isin(candidateVideos)]
print(len(df))
df

43


,vidId,publishTime,vidTitle,duration,viewCount,likeCount,commentCount,topicCategories,description
UCsDmESjqNPukDmVnuneLrqw,EcgkRp2IUsc,2017-10-26T01:05:22Z,My Hot Babysitter Got Me In Trouble,PT6M30S,35960829,840561,75550,"[https://en.wikipedia.org/wiki/Entertainment, ...",First Name Babysitter. Last Name Trouble. \nGE...
UCY6Ij8zOds0WJEeqCLOnqOQ,kbCah6yhYRs,2019-06-11T11:00:07Z,Descendants doesn&#39;t make any sense...,PT16M8S,9850298,263259,22459,"[https://en.wikipedia.org/wiki/Entertainment, ...",Descendants doesn't make any sense animation\n...
UClN24S5-fc7LD0JJW0FdGWw,ewsGmhAjjjI,2018-10-09T20:50:50Z,I have APHANTASIA (and you may too...without r...,PT12M54S,6041764,320113,72063,"[https://en.wikipedia.org/wiki/Hobby, https://...",So...yeah. I'm an artist that can't see pictur...
UC3iNdSV_RQU7DHTGpqEW96w,gA0bi-bFEYs,2018-06-22T19:20:36Z,How KSI Saved Logan Paul&#39;s Career,PT6M48S,20326487,514050,95645,"[https://en.wikipedia.org/wiki/Entertainment, ...",This KSI vs Logan Paul fight is interesting to...
UC_52i6mlrzOZs_NEuSepzAg,so1_5hYUEE8,2017-09-20T22:12:54Z,Being a Boba Barista (Work Stories),PT5M20S,2196765,58187,3756,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,Get it? “B”eing a “B”oba “B”arista “b”y “B”ill...
UCgCKYs56-LKEPGQ99DzqQOg,W8P5ewPk9fM,2019-01-18T20:00:06Z,Why I Dont Want a Hot Wife (Ft. Emirichu),PT8M7S,2319758,115467,5838,[https://en.wikipedia.org/wiki/Entertainment],"This week, we talk about one of the things I g..."
UC2Ds30pkifFVD0CE08wF50g,Mv8OkBjySGQ,2017-09-15T00:08:53Z,My Super HOT English Teacher (Animation),PT8M5S,9256911,303955,26316,"[https://en.wikipedia.org/wiki/Entertainment, ...","I have so many more stories, so subscribe and ..."
UC7UGbBVrqLlq6CRxWWUmyKw,2juKkLxdQo0,2019-05-11T17:14:40Z,"By the way, Are You a Sociopath?",PT11M23S,15162571,587421,45079,"[https://en.wikipedia.org/wiki/Entertainment, ...",Dragon City Sponsor link here: https://spons....
UCn1XB-jvmd9fXMzhiA6IR0w,eNGgfPs0Xp8,2016-09-26T21:49:18Z,Crushes,PT7M27S,31840209,912487,86962,"[https://en.wikipedia.org/wiki/Entertainment, ...",Get Crunchyroll Premium Ad Free for 30 Days: h...
UCFeqAfEuKm7lIg2ddQzh61A,rlSXaDq3uOk,2019-06-29T18:02:32Z,Childhood Crushes 2,PT17M16S,23048562,757408,83021,"[https://en.wikipedia.org/wiki/Entertainment, ...",Get Honey for FREE and start saving money toda...


Get lineart for all frames

couple that do not fit the criteria:: vidIds [kQEtRoyFfI8- gameplay, hpQQohcHk9Q- too short. not commentary, 6wS_uON5s6Q- speeddrawing, RlU32AfEVeU- footage apparently from another youtuber, iVqhzEaJhDw- too short, Xnv7JGqjaAo- speeddraw, iRBmUQQzpWQ- simply not animation, 18msRdBF11A- not animation, Ln4AnsWNUQI- not animation, Hkz0NcKPzMs- too short]



The plan is to find the distribution of DIFF. It'll vary based on the artist.

In [52]:
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from IPython.display import Image, display

dictOfVidIdToDiffMse = {}

def getFeatureVectorOfImage_color (imagePath):
    image = cv2.imread(imagePath)
    vectors = []
    for i, col in enumerate(['b', 'g', 'r']):
        hist = cv2.calcHist([image], [i], None, [256], [0, 256])
        vectors.append(hist)
        # plt.plot(hist, color = col)
        # plt.xlim([0, 256])
    # plt.show()
    vectors = np.vstack(vectors)
    return vectors

def getFeatureVectorOfImage (imagePath):
    image = cv2.imread(imagePath)
    vectors = []
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    vectors = np.vstack(hist)
    return vectors

def MSE_images (vA, vB):
    return np.sum(np.square(np.subtract(vA, vB)))

def DIFF_images(vA, vB):
    return (np.abs(vA - vB)).mean()

def removeLowDiff_andDemo(folderName):
    vidId = folderName.split('/')[-1]
    print(folderName)
    frameList = glob.glob(folderName+'/*.png') #unfortunately, doesn't work bc sortByInt isn't a thing
    filesMoved = 0 
    validFrameList = []

    lowRangeDone= False
    midRangeDone= False
    highRangeDone= False

    listOfMSE = []
    listOfDIFF = []

    imageShape = cv2.imread(frameList[0]).shape
    h,w,c = imageShape
    for i in range(len(frameList)):
        firstFrame = '{}/{}.png'.format(folderName, i)
        secondFrame = '{}/{}.png'.format(folderName, i+1)
        f1_v = getFeatureVectorOfImage_color(firstFrame)
        f2_v = getFeatureVectorOfImage_color(secondFrame)
        diff = DIFF_images(f1_v, f2_v)
        mse = MSE_images(f1_v, f2_v)/ (h*w)
        listOfDIFF.append(diff)
        listOfMSE.append(mse)

        if (diff >= 2):
            validFrameList.append(firstFrame)
            filesMoved+=1
        # if (lowRangeDone and midRangeDone and highRangeDone) == False:
        #     if (diff == 0 and not lowRangeDone):
        #         print ("low diff = : ", diff)
        #         print(firstFrame)
        #         display(Image(filename=firstFrame))
        #         print (secondFrame)
        #         display(Image(filename=secondFrame))
        #         lowRangeDone = True
        #     if (diff > 100 and diff < 150  and not midRangeDone):
        #         print ("mid diff = : ", diff)
        #         print(firstFrame)
        #         display(Image(filename=firstFrame))
        #         print (secondFrame)
        #         display(Image(filename=secondFrame))
        #         midRangeDone = True
        #     if (diff > 250 and diff < 300  and not highRangeDone):
        #         print ("high diff = : ", diff)
        #         print(firstFrame)
        #         display(Image(filename=firstFrame))
        #         print (secondFrame)
        #         display(Image(filename=secondFrame))
        #         highRangeDone = True
    print(len(frameList) - filesMoved, "out of", len(frameList), "files were duplicates")

    dictOfVidIdToDiffMse[vidId] = {'validFrameList': validFrameList, 'diffMin': str(np.amin(listOfDIFF)), 'diffMax': str(np.amax(listOfDIFF)), 'diffMean': str(np.mean(listOfDIFF)), 
                                    'mseMin': str(np.amin(listOfMSE)), 'mseMax': str(np.amax(listOfMSE)), 'mseMean': str(np.mean(listOfMSE))}
    return validFrameList
            

for lineartPath in glob.glob("frames/originals/*"):
    if lineartPath.split('/')[-1] not in not_candidate_vidIds:
        removeLowDiffDst = removeLowDiff_andDemo(lineartPath)
    print("===============================================================================")

frames/originals/Mv8OkBjySGQ


error: OpenCV(4.5.2) ../modules/imgproc/src/histogram.cpp:1293: error: (-215:Assertion failed) csz == 0 || csz == dims in function 'calcHist'


In [37]:
with open('vidIdToFramesListAndStats.json', 'w') as output:
    json.dump(dictOfVidIdToDiffMse, output)

In [40]:
dictOfVidIdToDiffMse

{'Mv8OkBjySGQ': {'validFrameList': ['frames/lineart/Mv8OkBjySGQ/9999.png',
   'frames/lineart/Mv8OkBjySGQ/10000.png',
   'frames/lineart/Mv8OkBjySGQ/10001.png',
   'frames/lineart/Mv8OkBjySGQ/10002.png',
   'frames/lineart/Mv8OkBjySGQ/10003.png',
   'frames/lineart/Mv8OkBjySGQ/10004.png',
   'frames/lineart/Mv8OkBjySGQ/10005.png',
   'frames/lineart/Mv8OkBjySGQ/10006.png',
   'frames/lineart/Mv8OkBjySGQ/10007.png',
   'frames/lineart/Mv8OkBjySGQ/10008.png',
   'frames/lineart/Mv8OkBjySGQ/10009.png',
   'frames/lineart/Mv8OkBjySGQ/10010.png',
   'frames/lineart/Mv8OkBjySGQ/10011.png',
   'frames/lineart/Mv8OkBjySGQ/10012.png',
   'frames/lineart/Mv8OkBjySGQ/10013.png',
   'frames/lineart/Mv8OkBjySGQ/10014.png',
   'frames/lineart/Mv8OkBjySGQ/10015.png',
   'frames/lineart/Mv8OkBjySGQ/10016.png',
   'frames/lineart/Mv8OkBjySGQ/10017.png',
   'frames/lineart/Mv8OkBjySGQ/10018.png',
   'frames/lineart/Mv8OkBjySGQ/10019.png',
   'frames/lineart/Mv8OkBjySGQ/10020.png',
   'frames/lineart/Mv8

In [50]:
for key, value in dictOfVidIdToDiffMse.items():
    if value['diffMean']!='0.0':
        print(key, value['diffMean'], value['mseMean'])

Mv8OkBjySGQ 3.0271358 72.44832085788107
2yFCyPX3kT0 1.5083073 96.50090819328233
OyDLuom4KGs 2.7417219 28.99979361223354
2juKkLxdQo0 1.9347183 72.68242656171881
uDvPIKM-L1o 7.802689 200.06487497676147
2bGkEK8I6zQ 4.075457 215.04496627236261
IwxWmKsVR5U 0.84233046 27.356301389828722
_uk_6vfqwTA 7.682624 174.00534080701402
cZ_CnLE6SPo 4.356926 370.7936619841624
Vm6Yu2N-ePI 1.9315648 211.65283172347424
O0hyjRF6quc 2.811688 70.86563525161384
5pMckBGWzAY 9.776351 169.1687493779067
0Vxp_Lj2b-E 1.1204095 47.768037206135226
A6V1QujNz8s 2.7934015 135.33856225660995
eNGgfPs0Xp8 0.2705184 32.738057132290756
rlSXaDq3uOk 2.8935428 41.22134298369679
1lME1ARYMBI 6.123885 97.8925342193273
PgajWuZA408 4.3425617 228.0865444111928
F8A-tXp09fs 2.265606 26.144847567789036
plSyrHqUh78 8.178008 475.17370576181105
Y7lYeRqhQ9Q 3.7197387 370.23196925682566
kbCah6yhYRs 8.734703 359.57555408614684
0TlV3w1YGqk 0.2841474 18.189149033909224
7WSo1Uw-p_g 0.28786027 27.01338080879927
P2EjH7l_N70 2.5212474 203.9854471901

In [135]:
def detectScenes(folderName, diff):
    with open(folderName+"_sceneDetection.txt", 'w') as f:
        sortedFrameList = sorted(glob.glob(folderName+'*.png'), key=os.path.getmtime)
        startOfShot=0
        numberOfScenes = 0
        lengthOfScene = 0
        for index in len(sortedFrameList-1):
            if MSE_images(sortedFrameList[index], sortedFrameList[index+1]) > diff:
                if lengthOfScene > 8:
                    f.write(startOfShot, ", ", index)
                    numberOfScenes += 1
                startOfShot = index+1
                lengthOfScene = 0
